<a href="https://colab.research.google.com/github/amura/DetectingDeepFakes/blob/faceprocessing/DetectingDeepFakesPart1_faceprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring dataset

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import pandas as pd
from io import BytesIO
import numpy as np
import os
import shutil
import pprint
import json
from PIL import Image
from PIL.ImageStat import Stat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import pathlib
import cv2

In [2]:
# mmove images to folder for use later
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/deepfakes


/content/gdrive/MyDrive/deepfakes


### face detection

In [5]:
# install MTCNN
!pip install mtcnn 


     |████████████████████████████████| 2.3MB 13.8MB/s 


Function for extracting the coordinates for the top left bounding box, width and height, and location of eyes

In [6]:
from mtcnn import MTCNN
import cv2

class DeepFakesDetector:
   # desired left eye location:  common to see percentages within the range of 20-40%. This controls the amount of face visible in final image
   def __init__(self,  desired_size=256, desired_lefteye_location=(0.35,0.35)):
      self.desired_size = desired_size
      self.desired_lefteye_location = desired_lefteye_location

  # this extracts the bounding box as well as the eye location
   @staticmethod
   def extractFaceFromImage( filePath, faceIndex = 0):
      #detect face
      img = cv2.cvtColor(cv2.imread(filePath), cv2.COLOR_BGR2RGB)
      detector = MTCNN()
      result = detector.detect_faces(img)

      # assumes only single person in the picture so index is 0
      return  result[faceIndex]['box'],  result[faceIndex]['keypoints']['left_eye'],  result[faceIndex]['keypoints']['right_eye'], img
       
   def crop_and_align(self, img, left, right):
      angle, dX, dY = self.eyes_angle(left, right)
      
      #work out scaling factor
      self.scaling_factor(dX, dY)

      mat = self.rotation_matrix(angle, left, right)

      return self.transform(img, mat)

  # This gets the angle between the eyes
   def eyes_angle(self,left, right):
      dY = right[1] - left[1]
      dX = right[0] - left[0]
      return np.degrees(np.arctan2(dY, dX)), dX, dY


   def scaling_factor(self, dX, dY):
      desiredRightEyeX = 1.0 - self.desired_lefteye_location[0]
      hyp = np.sqrt((dX ** 2) + (dY ** 2))
      desiredDist = (desiredRightEyeX - self.desired_lefteye_location[0])
      desiredDist *= self.desired_size
      self.scale = desiredDist / hyp

   def rotation_matrix(self, angle, left, right):
      # compute center (x, y)-coordinates (i.e., the median point)
      # between the two eyes in the input image
      eyesCenter = ((left[0] + right[0]) // 2,  (left[1] + right[1]) // 2)

      # grab the rotation matrix for rotating and scaling the face
      M = cv2.getRotationMatrix2D(eyesCenter, angle, self.scale)
         
      # update the translation component of the matrix
      tX = self.desired_size * 0.5
      tY = self.desired_size * self.desired_lefteye_location[1]
      M[0, 2] += (tX - eyesCenter[0])
      M[1, 2] += (tY - eyesCenter[1])     

      return M

   def transform(self, image, M):
      # apply the affine transformation
      #(w,h) = (self.desired_size, self.desired_size)
		  return cv2.warpAffine(image, M, (self.desired_size, self.desired_size),flags=cv2.INTER_CUBIC)
      


In [ ]:
from google.colab.patches import cv2_imshow

bb, left, right, img = DeepFakesDetector.extractFaceFromImage('./deepfake_sa1_output/00001.jpg')

In [ ]:
det = DeepFakesDetector()
alignedFace = det.crop_and_align(img,left, right)
cv2_imshow(cv2.cvtColor(alignedFace, cv2.COLOR_RGB2BGR))
cv2_imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [10]:
# !rm -rf ./deepfake_sa1_processed/
# !rm -rf ./real_sa1_processed/

In [11]:
import glob
import os

##setup output folders
deepFakeProcessed = './deepfake_sa1_processed'
realProcessed = './real_sa1_processed'
try:
      os.makedirs(deepFakeProcessed)
except OSError:
      pass

try:
      os.makedirs(realProcessed)
except OSError:
      pass

try:
      #make sure folders are clean
      !rm -rf ./deepfake_sa1_processed/
      !rm -rf ./real_sa1_processed/      
except OSError:
      pass

det = DeepFakesDetector()
# process all faces in directory and save as png
for imgPath in glob.glob('./deepfake_sa1_output/*.jpg'):
    baseFileName = os.path.basename(imgPath)
    outputFileName = os.path.join(deepFakeProcessed,os.path.splitext(baseFileName)[0] + '.png')      
    bb, left, right, img = DeepFakesDetector.extractFaceFromImage(imgPath)  
    alignedFace = det.crop_and_align(img,left, right)      
    cv2.imwrite(outputFileName, cv2.cvtColor(alignedFace, cv2.COLOR_RGB2BGR))

for imgPath in glob.glob('./real_sa1_output/*.jpg'):
    baseFileName = os.path.basename(imgPath)
    outputFileName = os.path.join(realProcessed,os.path.splitext(baseFileName)[0] + '.png')      
    bb, left, right, img = DeepFakesDetector.extractFaceFromImage(imgPath)  
    alignedFace = det.crop_and_align(img,left, right)      
    cv2.imwrite(outputFileName, cv2.cvtColor(alignedFace, cv2.COLOR_RGB2BGR))    
  